In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))

In [2]:
# saved files testing
from src.utils.flow import *
from src.utils.flow_calculations import *
from src.utils.restoration import *
from src.utils.truncated_packet import *
from src.operations.size_perturbation_logic import *
from src.operations.timing_perturbation_logic import *

modified_pcap_path = "../data/interim/testing_small_perturbed.pcapng"
pcap_file_path = '../data/raw/testing_small.pcapng'

In [25]:
truncated_packets = create_truncated_packets_from_pcap(pcap_file_path)
truncated_packets = assign_flow_ids_to_packets(truncated_packets)
flow_id = 2

### operations

1) size - infuences timing results so has to be first
   - uniform 
     - total scaling (+/-) (*)
   - normal
     - normal scaling with mean&std (+)

  
2) time - doesn't influence size results
   - uniform
     - total scaling (+/-) (*)
   - normal
     - normal scaling with mean&std (+)

Optimization phases:
1) optimize size with sequence of steps to minimize fitness error, direction should be taken into consideration, output should be the best possible obtained result for each flow; 
        fitness attributes: "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts",
        "fwd_pkt_len_max", "fwd_pkt_len_min", "fwd_pkt_len_mean", "fwd_pkt_len_std",
        "bwd_pkt_len_max", "bwd_pkt_len_min", "bwd_pkt_len_mean", "bwd_pkt_len_std",
        "pkt_len_mean", "pkt_len_std",
2) optimize timestamp with sequence of steps to minimize fitness error, directoin can't be taken into consideration dute to timing constrains, optimizing each direction independently would mess up the order; 
        fitness attributes: "flow_iat_mean", "flow_iat_max", "flow_iat_min", "flow_iat_std",
        "fwd_iat_tot", "fwd_iat_max", "fwd_iat_min", "fwd_iat_mean", "fwd_iat_std",
        "bwd_iat_tot", "bwd_iat_max", "bwd_iat_min", "bwd_iat_mean", "bwd_iat_std",
3) optimize ack&syn flags as much as possible?

Operation space:
Size - normal:
adjust_packet_size(truncated_packets, flow_id, direction, method='uniform', scaling_factor, focus_point)
pocus_point={"start", "middle", "end"}
scaling_factor = [1.1, 5.0]

Size - uniform:
adjust_packet_size(truncated_packets, flow_id, method="uniform", scaling_factor, direction)
scaling_factor = [0.25, 10.0]
direction = {0, 1, 2}

Time - normal:
apply_time_perturbation_with_focus(truncated_packets, flow_id, focus_point, method="normal", scaling_factor)
pocus_point={"start", "middle", "end"}
scaling_factor=[1.1, 3.0]

Time - uniform:
apply_time_perturbation_with_focus(truncated_packets, flow_id, method="uniform", scaling_factor)
scaling_factor=[0.4, 2.4]


Fitness function:
Calculate_MAD_sizing(truncated_packets, flow_id)
Calculate_MAD_timing(truncated_packets, flow_id)


In [34]:
# target dataset import
import pandas as pd

target_dataset = pd.read_csv('../data/processed/CTU13_Normal_Traffic_20.csv')
additional_test = pd.read_csv('../data/processed/CTU13_Normal_Traffic_80.csv')

In [ ]:
# TODO: standaryzacja, tam w ogole nie wykonuje sie taka loguka tego bledu jaka miala
# TODO: weryfikacja bo coś tam się chyba nie zgadza a to bardzo wazne

In [27]:
# sizing

def prepare_size_stats(truncated_packets, flow_id):
    
    size_stats = calculate_size_statistics(flow_id, truncated_packets, direction=0)
    size_stats_fwd = calculate_size_statistics(flow_id, truncated_packets, direction=1)
    size_stats_bwd = calculate_size_statistics(flow_id, truncated_packets, direction=2)
    
    total_fwd_pkts = total_packet_count(flow_id, truncated_packets, direction=1)
    total_bwd_pkts = total_packet_count(flow_id, truncated_packets, direction=2)

    adjusted_size_stats = {
        "tot_fwd_pkts": total_fwd_pkts,
        "tot_bwd_pkts": total_bwd_pkts,
        "totlen_fwd_pkts": size_stats_fwd['mean'] * total_fwd_pkts,
        "totlen_bwd_pkts": size_stats_bwd['mean'] * total_bwd_pkts,
        "fwd_pkt_len_max": size_stats_fwd['max'],
        "fwd_pkt_len_min": size_stats_fwd['min'],
        "fwd_pkt_len_mean": size_stats_fwd['mean'],
        "fwd_pkt_len_std": size_stats_fwd['std'],
        "bwd_pkt_len_max": size_stats_bwd['max'],
        "bwd_pkt_len_min": size_stats_bwd['min'],
        "bwd_pkt_len_mean": size_stats_bwd['mean'],
        "bwd_pkt_len_std": size_stats_bwd['std'],
        "pkt_len_mean": size_stats['mean'],
        "pkt_len_std": size_stats['std'],
    }

    return adjusted_size_stats



In [28]:
def calculate_mad_sizing(adjusted_size_stats, target_dataset):
    mad_values = []
    for stat in adjusted_size_stats:
        if stat in target_dataset:
            mad_value = abs(adjusted_size_stats[stat] - target_dataset[stat])
            mad_values.append(mad_value)
            print(mad_value)
    
    return np.mean(mad_values) if mad_values else np.nan


In [29]:
adjusted_size_stats = prepare_size_stats(truncated_packets, flow_id)
mad_sizing = calculate_mad_sizing(adjusted_size_stats, target_dataset)

0        1
1        1
2        1
3        1
4        1
        ..
10658    0
10659    1
10660    1
10661    1
10662    1
Name: tot_fwd_pkts, Length: 10663, dtype: int64
0        1
1        1
2        1
3        1
4        1
        ..
10658    2
10659    1
10660    1
10661    1
10662    1
Name: tot_bwd_pkts, Length: 10663, dtype: int64
0        350.0
1        350.0
2        350.0
3        350.0
4        350.0
         ...  
10658    311.0
10659    350.0
10660    350.0
10661    350.0
10662    350.0
Name: totlen_fwd_pkts, Length: 10663, dtype: float64
0        430.0
1        418.0
2        590.0
3        508.0
4        366.0
         ...  
10658    295.0
10659    394.0
10660    590.0
10661    440.0
10662    508.0
Name: totlen_bwd_pkts, Length: 10663, dtype: float64
0        350
1        350
2        350
3        350
4        350
        ... 
10658    311
10659    350
10660    350
10661    350
10662    350
Name: fwd_pkt_len_max, Length: 10663, dtype: int64
0        350
1        350
2     

In [13]:
mad_sizing

3548.521898093314

In [39]:
additional_test.head(1)

,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,fwd_pkt_len_std,bwd_pkt_len_max,bwd_pkt_len_min,...,fwd_iat_std,bwd_iat_tot,bwd_iat_max,bwd_iat_min,bwd_iat_mean,bwd_iat_std,fin_flag_cnt,syn_flag_cnt,rst_flag_cnt,ack_flag_cnt
0,0,2,0,163,0,0,0.0,0.0,114,49,...,0.0,39686,39686,39686,39686.0,0.0,0,0,0,0


In [40]:
mad_sizing = calculate_mad_sizing(additional_test.head(1), target_dataset)
mad_sizing

0        0.0
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10658    NaN
10659    NaN
10660    NaN
10661    NaN
10662    NaN
Name: tot_fwd_pkts, Length: 10663, dtype: float64
0        0.0
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10658    NaN
10659    NaN
10660    NaN
10661    NaN
10662    NaN
Name: tot_bwd_pkts, Length: 10663, dtype: float64
0        0.0
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10658    NaN
10659    NaN
10660    NaN
10661    NaN
10662    NaN
Name: totlen_fwd_pkts, Length: 10663, dtype: float64
0        3.0
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10658    NaN
10659    NaN
10660    NaN
10661    NaN
10662    NaN
Name: totlen_bwd_pkts, Length: 10663, dtype: float64
0        0.0
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10658    NaN
10659    NaN
10660    NaN
10661    NaN
10662    NaN
Name: fwd_pkt_len_max, Length: 10663, dtype: float64
0        0.0
1        NaN


nan

In [ ]:
testing_dataset.

In [31]:
# timing
def get_timing_stats_for_mad(truncated_packets, flow_id):
    """
    Obtain and adjust timing statistics for a specific flow ID from truncated packets,
    considering the entire flow as well as forward and backward directions separately.
    """
    # Obtain timing stats for the entire flow (direction=0)
    timing_stats_total = calculate_delta_time_statistics(flow_id, truncated_packets, direction=0)
    # Obtain timing stats for the forward direction (fwd, direction=1)
    timing_stats_fwd = calculate_delta_time_statistics(flow_id, truncated_packets, direction=1)
    # Obtain timing stats for the backward direction (bwd, direction=2)
    timing_stats_bwd = calculate_delta_time_statistics(flow_id, truncated_packets, direction=2)

    # Combining and adjusting timing stats to the expected format for MAD calculation
    adjusted_timing_stats = {
        "flow_iat_mean": timing_stats_total.get('mean', 0),
        "flow_iat_max": timing_stats_total.get('max', 0),
        "flow_iat_min": timing_stats_total.get('min', 0),
        "flow_iat_std": timing_stats_total.get('std', 0),
        "fwd_iat_tot": timing_stats_fwd.get('total', 0),
        "fwd_iat_max": timing_stats_fwd.get('max', 0),
        "fwd_iat_min": timing_stats_fwd.get('min', 0),
        "fwd_iat_mean": timing_stats_fwd.get('mean', 0),
        "fwd_iat_std": timing_stats_fwd.get('std', 0),
        "bwd_iat_tot": timing_stats_bwd.get('total', 0),
        "bwd_iat_max": timing_stats_bwd.get('max', 0),
        "bwd_iat_min": timing_stats_bwd.get('min', 0),
        "bwd_iat_mean": timing_stats_bwd.get('mean', 0),
        "bwd_iat_std": timing_stats_bwd.get('std', 0),
    }

    return adjusted_timing_stats


In [32]:
def calculate_mad_timing(timing_stats, target_dataset):
    mad_values = []
    for stat in timing_stats:
        if stat in target_dataset:
            mad_value = abs(timing_stats[stat] - target_dataset[stat])
            mad_values.append(mad_value)
            print(mad_value)
    
    return np.mean(mad_values) if mad_values else np.nan


In [33]:
timing_stats = get_timing_stats_for_mad(truncated_packets, flow_id)
mad_timing = calculate_mad_timing(timing_stats, target_dataset)

AttributeError: 'str' object has no attribute 'get'

In [24]:
mad_timing

2977765.55457922